In [ ]:
# ==============================
# PART 4: EXPLAINABLE AI (FAST VERSION)
# AI Drone Anomaly Detection
# ==============================

# Install packages if running in Colab
!pip install --quiet shap lime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap
from lime.lime_tabular import LimeTabularExplainer
import joblib
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.inspection import PartialDependenceDisplay

sns.set(style="whitegrid")

# ------------------------------
# Step 1: Load preprocessed dataset
# ------------------------------
data = pd.read_csv('/content/sample_data/drone_data_preprocessed.csv')
X = data.drop('Label', axis=1).values
y = data['Label'].values
feature_names = data.drop('Label', axis=1).columns

# Train-test split (match fast version subset)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, train_size=0.3, random_state=42, stratify=y)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ------------------------------
# Step 2: Load models
# ------------------------------
xgb_model = joblib.load("xgb_model_fast.pkl")
fnn_model = load_model("fnn_model_fast.h5")

# ------------------------------
# Step 3: XGBoost Feature Importance
# ------------------------------
xgb_importance = xgb_model.best_estimator_.feature_importances_
xgb_feat_df = pd.DataFrame({'Feature': feature_names, 'Importance': xgb_importance}).sort_values(by='Importance', ascending=False)

plt.figure(figsize=(10,6))
sns.barplot(x='Importance', y='Feature', data=xgb_feat_df.head(15))
plt.title("Top 15 XGBoost Feature Importances")
plt.show()

# ------------------------------
# Step 4: SHAP Analysis
# ------------------------------
# --- 4a: XGBoost SHAP ---
explainer_xgb = shap.TreeExplainer(xgb_model.best_estimator_)
shap_values_xgb_all = explainer_xgb.shap_values(X_test)

class_index = 1
shap_values_xgb = shap_values_xgb_all[class_index]
shap.summary_plot(shap_values_xgb, X_test, feature_names=feature_names, plot_type='bar')

# SHAP dependence plot for top 3 features
top_features = xgb_feat_df['Feature'].head(3).tolist()
for feat in top_features:
    shap.dependence_plot(feat,shap_values_xgb,X_test,feature_names=feature_names)

# --- 4b: FNN SHAP ---
# Use small background for speed
background = X_train_scaled[np.random.choice(X_train_scaled.shape[0], 50, replace=False)]

def fnn_predict(X):
    return fnn_model.predict(X)

explainer_fnn = shap.KernelExplainer(fnn_predict, background)
shap_values_fnn = explainer_fnn.shap_values(X_test_scaled[:50], nsamples=100)

shap.summary_plot(shap_values_fnn, X_test_scaled[:50], feature_names=feature_names, plot_type='bar')

# ------------------------------
# Step 5: LIME Explanations (FNN)
# ------------------------------
explainer_lime = LimeTabularExplainer(
    X_train_scaled,
    feature_names=feature_names,
    class_names=[str(c) for c in np.unique(y_train)],
    discretize_continuous=True
)

# Explain 3 sample predictions
for i in range(3):
    exp = explainer_lime.explain_instance(X_test_scaled[i], fnn_model.predict)
    print(f"LIME explanation for sample {i}")
    exp.show_in_notebook(show_table=True)

# ------------------------------
# Step 6: Partial Dependence Plots (Top Features)
# ------------------------------
top_features = xgb_feat_df['Feature'].head(5).tolist()
PartialDependenceDisplay.from_estimator(
    xgb_model.best_estimator_,
    X_test,
    features=top_features,
    feature_names=feature_names,
    grid_resolution=20
)
plt.show()

print("FAST VERSION: XAI analysis complete!")